In [1]:

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from pygrinder import mcar
from pypots.data import load_specific_dataset

I:\work\SANNI\venv\lib\site-packages\pypots\nn\modules\reformer\local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
I:\work\SANNI\venv\lib\site-packages\pypots\nn\modules\reformer\local_attention.py:102: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


In [3]:
data = load_specific_dataset('physionet_2012')  # PyPOTS will automatically download and extract it.


2024-09-13 17:31:11 [INFO]: Loading the dataset physionet_2012 with TSDB (https://github.com/WenjieDu/Time_Series_Data_Beans)...
2024-09-13 17:31:11 [INFO]: Starting preprocessing physionet_2012...
2024-09-13 17:31:11 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2024-09-13 17:31:11 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2024-09-13 17:31:11 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2024-09-13 17:31:11 [INFO]: Loaded successfully!
2024-09-13 17:31:49 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2024-09-13 17:31:49 [INFO]: 69238 values masked out in the val set as ground truth, take 10.03% of the original observed values
2024-09

In [4]:
data.keys()

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'val_X_ori', 'test_X_ori'])

In [8]:
X = data['train_X']
num_samples = len(X)
# X = X.drop(['RecordID', 'Time'], axis = 1)
X = X.reshape(-1,37)
X = StandardScaler().fit_transform(X)
X = X.reshape(num_samples, 48, -1)
X_ori = X  # keep X_ori for validation
X = mcar(X, 0.1)

In [9]:
print(X.shape)  # (11988, 48, 37), 11988 samples and each sample has 48 time steps, 37 features

(7671, 48, 37)


In [11]:
from pypots.imputation import BRITS
from BRITS.PypotsBrits import BRITS as myBrits

In [12]:
saits = BRITS(n_steps=48, n_features=37, epochs=10,rnn_hidden_size=10)


2024-09-13 17:33:12 [INFO]: No given device, using default device: cuda
2024-09-13 17:33:12 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-09-13 17:33:17 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,628


In [13]:
dataset = {"X": X[:100]}  # X for model input
saits.fit(dataset)  # train the model on the dataset


2024-09-13 17:33:27 [INFO]: Epoch 001 - training loss: 1.4454
2024-09-13 17:33:30 [INFO]: Epoch 002 - training loss: 1.4355
2024-09-13 17:33:33 [INFO]: Epoch 003 - training loss: 1.3529
2024-09-13 17:33:36 [INFO]: Epoch 004 - training loss: 1.3823
2024-09-13 17:33:39 [INFO]: Epoch 005 - training loss: 1.5099
2024-09-13 17:33:42 [INFO]: Epoch 006 - training loss: 1.3715
2024-09-13 17:33:45 [INFO]: Epoch 007 - training loss: 1.3376
2024-09-13 17:33:48 [INFO]: Epoch 008 - training loss: 1.3605
2024-09-13 17:33:51 [INFO]: Epoch 009 - training loss: 1.2924
2024-09-13 17:33:55 [INFO]: Epoch 010 - training loss: 1.3248
2024-09-13 17:33:55 [INFO]: Finished training. The best model is from epoch#9.


In [101]:
dataset = {"X": X[:100]}  # X for model input
myBritrs = myBrits(n_steps=48, n_features=37, epochs=10,rnn_hidden_size=10)
myBritrs.fit(dataset) 

2024-09-13 22:49:47 [INFO]: No given device, using default device: cuda
2024-09-13 22:49:47 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2024-09-13 22:49:47 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 19,628


tensor(1.4751, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3448, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4243, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4016, device='cuda:0', grad_fn=<AddBackward0>)


2024-09-13 22:49:50 [INFO]: Epoch 001 - training loss: 1.4115


tensor(1.4403, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3572, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4114, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3025, device='cuda:0', grad_fn=<AddBackward0>)


2024-09-13 22:49:54 [INFO]: Epoch 002 - training loss: 1.3778


...

2024-09-13 22:50:12 [INFO]: Epoch 008 - training loss: 1.3129


tensor(1.2974, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3981, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2780, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.4077, device='cuda:0', grad_fn=<AddBackward0>)


2024-09-13 22:50:15 [INFO]: Epoch 009 - training loss: 1.3453


tensor(1.3088, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3022, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3466, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.3209, device='cuda:0', grad_fn=<AddBackward0>)


2024-09-13 22:50:19 [INFO]: Epoch 010 - training loss: 1.3196
2024-09-13 22:50:19 [INFO]: Finished training. The best model is from epoch#8.


In [77]:
brits_impute = saits.impute(dataset)

In [78]:
my_brits_impute = myBritrs.impute(dataset)
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)

tensor(0.7052, device='cuda:0') tensor(0.6363, device='cuda:0') tensor(1.3415, device='cuda:0')
tensor(0.6531, device='cuda:0') tensor(0.6347, device='cuda:0') tensor(1.2878, device='cuda:0')
tensor(0.5035, device='cuda:0') tensor(0.4907, device='cuda:0') tensor(0.9941, device='cuda:0')
tensor(0.6895, device='cuda:0') tensor(0.6528, device='cuda:0') tensor(1.3423, device='cuda:0')


In [79]:
indicating_mask = np.isnan(X) ^ np.isnan(X_ori)
indicating_mask=indicating_mask[:100]

In [80]:
from pypots.utils.metrics import calc_mae

calc_mae(brits_impute, np.nan_to_num(X_ori[:100]), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)


0.6425847256782081

In [98]:
calc_mae(my_brits_impute, np.nan_to_num(X_ori[:100]), indicating_mask)  # calculate mean absolute error on the ground truth (artificially-missing values)

0.6644782974148035

(100, 48, 37)

In [30]:
indicating_mask.shape

(7671, 48, 37)